In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
do_jsd = False
top_out_dir = top_evaluation_dir
boost_samples = 3
special_tag = ''
skip = ['test']

In [5]:
top_syn_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/'

In [6]:
# all_classes = ['airplane', 'car', 'table', 'sofa', 'rifle']
all_classes = ['chair']
best_model = {'gmm': 'bneck_128_full_32_gaussians', 'l_w_gan': 2000}
generators_zoo = ['l_w_gan']

In [7]:
gan_tag = '_mlp_with_split_1pc_usampled_bnorm_on_encoder_only_emd_bneck_128_lgan_arch'

In [8]:
do_mmd = True

In [9]:
for class_name in all_classes:    
    print class_name
    evaluator = Evaluator(class_name)
    evaluator.load_gt_data()
    if do_jsd:
        evaluator.prepare_gt_grid_variables()    
    for generator_name in generators_zoo:
        print generator_name
        sample_files = []
        if 'gan' in generator_name:
            top_sample_dir = osp.join(top_syn_dir, generator_name, generator_name + '_' + class_name + gan_tag)
            epoch = best_model[generator_name]
            sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))            
        elif 'gmm' in generator_name:
            gaussian_type = best_model[generator_name]
            sample_files.append(osp.join(top_syn_dir, 'gmm', 'gmm_emd_'+  class_name, gaussian_type + '.npz'))
        else:
            assert False

        for sample_file in sample_files:
            evaluator.prepare_sample_data(sample_file, boost_samples)
            sample_name = osp.basename(sample_file)[:-len('.npz')]
            print sample_name
        
        if do_jsd:
            evaluator.compute_jsd(skip=skip)
        if do_mmd:
            evaluator.compute_mmd(loss='chamfer', sample_estimator=False, n_samples=None, 
                              ref_pop_size=None, sample_pop_size=None, skip=skip)



chair
l_w_gan
epoch_2000
train 0.00151506 0.000889103
val 0.00197032 0.00108394
